In [1]:
#Import packages
import os
import openai
import gradio as gr
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Load the API key
openai.api_key = os.environ.get('API_KEY')

#Load Data
def load_data(ss_questions_filepath, faqs_filepath):
    ss_questions_data = pd.read_excel(ss_questions_filepath)
    faqs_data = pd.read_excel(faqs_filepath)
    default_question = ss_questions_data['question'].iloc[0]
    return ss_questions_data, faqs_data, default_question

# Load the data
ss_questions_filepath = 'final_updated_cleaned_SS_questions.xlsx'
faqs_filepath = 'ai_tutor_faqs.xlsx'
ss_questions_data, faqs_data, default_question = load_data(ss_questions_filepath, faqs_filepath)

#Define Helper Functions
class QueryHandler:
    
    def __init__(self, ss_questions_data, faqs_data):
        self.ss_questions_data = ss_questions_data
        self.faqs_data = faqs_data

    # Get FAQ response
    def get_faq_response(self, query):
        if query in self.faqs_data['FAQs'].values:
            return self.faqs_data[self.faqs_data['FAQs'] == query]['Answer'].iloc[0]
        else:
            return "Sorry, I couldn't find the requested information in the FAQs."
    
    # GPT live
    def get_completion_from_chat(self, messages):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            presence_penalty=1,
            frequency_penalty=1,
            max_tokens=3000
        )
        return response.choices[0].message["content"].strip()

    # Handle general questions
    def handle_general_query(self, query, request_type=None, further_clarification=False):
        system_message = "You are a programming assistant specialized in Python and data science. You should only answer questions related to these topics."
        if request_type == "Get Walkthrough":
            user_message = f"Provide a general walkthrough for the query: {query}"
        elif request_type == "Explain Solution":
            user_message = f"Explain the solution for the query: {query}"
        elif request_type == "Get Hint":
            user_message = f"Provide a hint for solving the query: {query}"
        else:
            user_message = f"Can you provide general information about the following topic: {query}"
        
        if further_clarification:
            user_message += "Please provide detailed steps and examples."
        
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
        
        response = self.get_completion_from_chat(messages)
        
        if "I can't assist with that request" in response or "I can't provide information on that topic" in response:
            return "The question doesn't seem to be related to Python or Data Science. Please provide a more specific question or choose from the provided options."
        
        return response

    # Get default answers from the database
    def get_initial_response(self, query, request_type):
        question_data = self.ss_questions_data[self.ss_questions_data['question'] == query].iloc[0].to_dict()
        if request_type == "Get Walkthrough":
            return question_data.get('python_hint', 'No walkthrough available.')
        elif request_type == "Get Solution":
            return question_data.get('python_solution', 'No explanation available.')
        elif request_type == "Get Hint":
            return question_data.get('hint', 'No hint available.')
        else:
            return "Unknown request."
    
    # Respond to the query
    def respond_to_query(self, input_text, dropdown_selection, request_type, further_clarification, *args):
        query = input_text if input_text and input_text != "Type your question or select from the dropdown." else dropdown_selection
        query = query or default_question

        if query in self.faqs_data['FAQs'].values:
            return self.get_faq_response(query)

        elif query in self.ss_questions_data['question'].values:
            if not further_clarification:
                return self.get_initial_response(query, request_type)
            else:
                system_message = "You are a helpful assistant."
                user_message = f"Please elaborate on the solution for the following problem. Provide steps of instructions and include an example of code in each step:{query}"
                messages = [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_message}
                ]
                return self.get_completion_from_chat(messages)
        else:
            return self.handle_general_query(query)

        

#Main Function and Interface
query_handler = QueryHandler(ss_questions_data, faqs_data)
app_launch = gr.Interface(
    fn=query_handler.respond_to_query,
    inputs=[
        gr.Textbox(default="Type your question or select from the dropdown.", type="text", label="Your Question"),
        gr.Dropdown(choices=faqs_data['FAQs'].tolist(), label="Or Select a FAQ Question"),
        gr.Radio(choices=["Get Walkthrough", "Get Solution", "Get Hint"], label="Request Type"),
        gr.Checkbox(label="Seek further clarification?", default=False)
    ],
    outputs=gr.Textbox(default="Chatbot is typing...", type="text", label="Output"),
    description=f"## Question: {default_question}",
    live=False,
    flagging=False,
    theme="dark",
    allow_flagging=False,
    title="AI Tutor"
)

app_launch.launch(share=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
